<a href="https://colab.research.google.com/github/ThisIsJorgeLima/DS-Unit-2-Linear-Models/blob/master/module2-regression-2/JAL_Dec_3_2019_assignment_regression_classification_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 1, Module 2*

---

# Regression 2

## Assignment

You'll continue to **predict how much it costs to rent an apartment in NYC,** using the dataset from renthop.com.

- [x] Do train/test split. Use data from April & May 2016 to train. Use data from June 2016 to test.
- [x] Engineer at least two new features. (See below for explanation & ideas.)
- [x] Fit a linear regression model with at least two features.
- [x] Get the model's coefficients and intercept.
- [ ] Get regression metrics RMSE, MAE, and $R^2$, for both the train and test data.
- [ ] What's the best test MAE you can get? Share your score and features used with your cohort on Slack!
- [x] As always, commit your notebook to your fork of the GitHub repo.


#### [Feature Engineering](https://en.wikipedia.org/wiki/Feature_engineering)

> "Some machine learning projects succeed and some fail. What makes the difference? Easily the most important factor is the features used." — Pedro Domingos, ["A Few Useful Things to Know about Machine Learning"](https://homes.cs.washington.edu/~pedrod/papers/cacm12.pdf)

> "Coming up with features is difficult, time-consuming, requires expert knowledge. 'Applied machine learning' is basically feature engineering." — Andrew Ng, [Machine Learning and AI via Brain simulations](https://forum.stanford.edu/events/2011/2011slides/plenary/2011plenaryNg.pdf) 

> Feature engineering is the process of using domain knowledge of the data to create features that make machine learning algorithms work. 

#### Feature Ideas
- Does the apartment have a description?
- How long is the description?
- How many total perks does each apartment have?
- Are cats _or_ dogs allowed?
- Are cats _and_ dogs allowed?
- Total number of rooms (beds + baths)
- Ratio of beds to baths
- What's the neighborhood, based on address or latitude & longitude?

## Stretch Goals
- [ ] If you want more math, skim [_An Introduction to Statistical Learning_](http://faculty.marshall.usc.edu/gareth-james/ISL/ISLR%20Seventh%20Printing.pdf),  Chapter 3.1, Simple Linear Regression, & Chapter 3.2, Multiple Linear Regression
- [ ] If you want more introduction, watch [Brandon Foltz, Statistics 101: Simple Linear Regression](https://www.youtube.com/watch?v=ZkjP5RJLQF4)
(20 minutes, over 1 million views)
- [ ] Add your own stretch goal(s) !

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Applied-Modeling/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'
    
# Ignore this Numpy warning when using Plotly Express:
# FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
import warnings
warnings.filterwarnings(action='ignore', category=FutureWarning, module='numpy')

In [0]:
import numpy as np
import pandas as pd

# Read New York City apartment rental listing data
df = pd.read_csv(DATA_PATH+'apartments/renthop-nyc.csv')
assert df.shape == (49352, 34)

# Remove the most extreme 1% prices,
# the most extreme .1% latitudes, &
# the most extreme .1% longitudes
df = df[(df['price'] >= np.percentile(df['price'], 0.5)) & 
        (df['price'] <= np.percentile(df['price'], 99.5)) & 
        (df['latitude'] >= np.percentile(df['latitude'], 0.05)) & 
        (df['latitude'] < np.percentile(df['latitude'], 99.95)) &
        (df['longitude'] >= np.percentile(df['longitude'], 0.05)) & 
        (df['longitude'] <= np.percentile(df['longitude'], 99.95))]

# **Assignment Starts Here**

# Do train/test split. 

In [0]:
from sklearn.metrics import mean_absolute_error
import plotly.express as px
from sklearn.linear_model import LinearRegression


In [0]:
df.shape

(48817, 34)

In [0]:
df.head()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
0,1.5,3,2016-06-24 07:54:24,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,Metropolitan Avenue,40.7145,-73.9425,3000,792 Metropolitan Avenue,medium,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1.0,2,2016-06-12 12:19:27,,Columbus Avenue,40.7947,-73.9667,5465,808 Columbus Avenue,low,1,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,1.0,1,2016-04-17 03:26:41,"Top Top West Village location, beautiful Pre-w...",W 13 Street,40.7388,-74.0018,2850,241 W 13 Street,high,0,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1.0,1,2016-04-18 02:22:02,Building Amenities - Garage - Garden - fitness...,East 49th Street,40.7539,-73.9677,3275,333 East 49th Street,low,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1.0,4,2016-04-28 01:32:41,Beautifully renovated 3 bedroom flex 4 bedroom...,West 143rd Street,40.8241,-73.9493,3350,500 West 143rd Street,low,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Trying something new...
# Finding new ways to explore data
df.head(2).T

,0,1
bathrooms,1.5,1
bedrooms,3,2
created,2016-06-24 07:54:24,2016-06-12 12:19:27
description,A Brand New 3 Bedroom 1.5 bath ApartmentEnjoy ...,
display_address,Metropolitan Avenue,Columbus Avenue
latitude,40.7145,40.7947
longitude,-73.9425,-73.9667
price,3000,5465
street_address,792 Metropolitan Avenue,808 Columbus Avenue
interest_level,medium,low


In [0]:
df.tail()

,bathrooms,bedrooms,created,description,display_address,latitude,longitude,price,street_address,interest_level,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
49347,1.0,2,2016-06-02 05:41:05,"30TH/3RD, MASSIVE CONV 2BR IN LUXURY FULL SERV...",E 30 St,40.7426,-73.9790,3200,230 E 30 St,medium,1,0,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
49348,1.0,1,2016-04-04 18:22:34,"HIGH END condo finishes, swimming pool, and ki...",Rector Pl,40.7102,-74.0163,3950,225 Rector Place,low,1,1,0,1,1,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1
49349,1.0,1,2016-04-16 02:13:40,Large Renovated One Bedroom Apartment with Sta...,West 45th Street,40.7601,-73.9900,2595,341 West 45th Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49350,1.0,0,2016-04-08 02:13:33,Stylishly sleek studio apartment with unsurpas...,Wall Street,40.7066,-74.0101,3350,37 Wall Street,low,1,1,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
49351,1.0,2,2016-04-12 02:48:07,Look no further!!! This giant 2 bedroom apart...,Park Terrace East,40.8699,-73.9172,2200,30 Park Terrace East,low,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
df.tail(2).T

,49350,49351
bathrooms,1,1
bedrooms,0,2
created,2016-04-08 02:13:33,2016-04-12 02:48:07
description,Stylishly sleek studio apartment with unsurpas...,Look no further!!! This giant 2 bedroom apart...
display_address,Wall Street,Park Terrace East
latitude,40.7066,40.8699
longitude,-74.0101,-73.9172
price,3350,2200
street_address,37 Wall Street,30 Park Terrace East
interest_level,low,low


In [0]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48817 entries, 0 to 49351
Data columns (total 34 columns):
bathrooms               48817 non-null float64
bedrooms                48817 non-null int64
created                 48817 non-null object
description             47392 non-null object
display_address         48684 non-null object
latitude                48817 non-null float64
longitude               48817 non-null float64
price                   48817 non-null int64
street_address          48807 non-null object
interest_level          48817 non-null object
elevator                48817 non-null int64
cats_allowed            48817 non-null int64
hardwood_floors         48817 non-null int64
dogs_allowed            48817 non-null int64
doorman                 48817 non-null int64
dishwasher              48817 non-null int64
no_fee                  48817 non-null int64
laundry_in_building     48817 non-null int64
fitness_center          48817 non-null int64
pre-war                 4

In [0]:
print(df.columns)

Index(['bathrooms', 'bedrooms', 'created', 'description', 'display_address',
       'latitude', 'longitude', 'price', 'street_address', 'interest_level',
       'elevator', 'cats_allowed', 'hardwood_floors', 'dogs_allowed',
       'doorman', 'dishwasher', 'no_fee', 'laundry_in_building',
       'fitness_center', 'pre-war', 'laundry_in_unit', 'roof_deck',
       'outdoor_space', 'dining_room', 'high_speed_internet', 'balcony',
       'swimming_pool', 'new_construction', 'terrace', 'exclusive', 'loft',
       'garden_patio', 'wheelchair_access', 'common_outdoor_space'],
      dtype='object')


In [0]:
df.describe()

,bathrooms,bedrooms,latitude,longitude,price,elevator,cats_allowed,hardwood_floors,dogs_allowed,doorman,dishwasher,no_fee,laundry_in_building,fitness_center,pre-war,laundry_in_unit,roof_deck,outdoor_space,dining_room,high_speed_internet,balcony,swimming_pool,new_construction,terrace,exclusive,loft,garden_patio,wheelchair_access,common_outdoor_space
count,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000,48817.000000
mean,1.201794,1.537149,40.750760,-73.972760,3579.585247,0.524838,0.478276,0.478276,0.447631,0.424852,0.415081,0.367085,0.052769,0.268452,0.185653,0.175902,0.132761,0.138394,0.102833,0.087203,0.060471,0.055206,0.051908,0.046193,0.043305,0.042711,0.039331,0.027224,0.026241
std,0.470711,1.106087,0.038954,0.028883,1762.430772,0.499388,0.499533,0.499533,0.497255,0.494326,0.492741,0.482015,0.223573,0.443158,0.388830,0.380741,0.339320,0.345317,0.303744,0.282136,0.238359,0.228385,0.221844,0.209905,0.203544,0.202206,0.194382,0.162738,0.159852
min,0.000000,0.000000,40.575700,-74.087300,1375.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,1.000000,40.728300,-73.991800,2500.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,1.000000,40.751700,-73.978000,3150.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,2.000000,40.774000,-73.955000,4095.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,10.000000,8.000000,40.989400,-73.700100,15500.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [0]:
print(df.mean())

NameError: ignored

In [0]:
print(df.bedrooms.value_counts())

In [0]:
df.sort_values('price', ascending=False).head()

In [0]:
# Extracting subsets
print(type(df.price))
print(df.price.head(5))

## Use data from April & May 2016 to train. Use data from June 2016 to test.

In [0]:
df.created

0        2016-06-24 07:54:24
1        2016-06-12 12:19:27
2        2016-04-17 03:26:41
3        2016-04-18 02:22:02
4        2016-04-28 01:32:41
                ...         
49347    2016-06-02 05:41:05
49348    2016-04-04 18:22:34
49349    2016-04-16 02:13:40
49350    2016-04-08 02:13:33
49351    2016-04-12 02:48:07
Name: created, Length: 48817, dtype: object

In [0]:
# Lets get rid of the time stamps

df['created'] = pd.to_datetime(df['created'], infer_datetime_format=True)

In [0]:
df['created'].describe()

count                   48817
unique                  48148
top       2016-05-14 01:11:03
freq                        3
first     2016-04-01 22:12:41
last      2016-06-29 21:41:47
Name: created, dtype: object

In [0]:
train = df[df['created'].dt.month < 6]
test = df[df['created'].dt.month == 6]

In [0]:
# Lets check how many observations are in the train and test set:
train.shape, test.shape

((31844, 34), (16973, 34))

# Engineer at least two new features.

In [0]:
# How many total perks does each apartment have?
# Lets get a list of all the ammenities related to perks column:
list(df.columns.values)

['bathrooms',
 'bedrooms',
 'created',
 'description',
 'display_address',
 'latitude',
 'longitude',
 'price',
 'street_address',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [0]:
# Lets train data with total perks:
perks = ['bathrooms',
 'bedrooms',
 'created',
 'description',
 'display_address',
 'latitude',
 'longitude',
 'price',
 'street_address',
 'interest_level',
 'elevator',
 'cats_allowed',
 'hardwood_floors',
 'dogs_allowed',
 'doorman',
 'dishwasher',
 'no_fee',
 'laundry_in_building',
 'fitness_center',
 'pre-war',
 'laundry_in_unit',
 'roof_deck',
 'outdoor_space',
 'dining_room',
 'high_speed_internet',
 'balcony',
 'swimming_pool',
 'new_construction',
 'terrace',
 'exclusive',
 'loft',
 'garden_patio',
 'wheelchair_access',
 'common_outdoor_space']

In [0]:
df['total_perks'] = df[perks].sum(axis=1)

In [0]:
# Ratio of beds to baths:
df ['bathroom_baths_ratio'] = df['bedrooms']/df['bathrooms']

In [0]:
print(train.isnull().sum())

bathrooms                 0
bedrooms                  0
created                   0
description             969
display_address          69
latitude                  0
longitude                 0
price                     0
street_address            6
interest_level            0
elevator                  0
cats_allowed              0
hardwood_floors           0
dogs_allowed              0
doorman                   0
dishwasher                0
no_fee                    0
laundry_in_building       0
fitness_center            0
pre-war                   0
laundry_in_unit           0
roof_deck                 0
outdoor_space             0
dining_room               0
high_speed_internet       0
balcony                   0
swimming_pool             0
new_construction          0
terrace                   0
exclusive                 0
loft                      0
garden_patio              0
wheelchair_access         0
common_outdoor_space      0
dtype: int64


In [0]:
# Lets get rid of infinite and NaNs
df ['bathroom_baths_ratio'] = df['bathroom_baths_ratio'].replace(np.inf, np.nan)
df ['bathroom_baths_ratio'].fillna(0, inplace = True)

# Begin with baselines

In [0]:
prices = df.price
print('Minimum $',prices.min())
print('Average $',prices.mean())
print('Maximum $',prices.max())

Minimum $ 1375
Average $ 3579.5852469426636
Maximum $ 15500


In [0]:
# What was the average price?
print('$',train['price'].mean())

$ 3575.604007034292


In [0]:
# Arrange y target vectors
target = 'price'
y_train = train[target]
y_test = test [target]

In [0]:
# Get mean baseline
# same as above ^
print('Mean Baseline (using 0 features):')
guess = y_train.mean()
print('$',(guess))

Mean Baseline (using 0 features):
$ total_perks             3553.774784
bathroom_baths_ratio       1.262440
dtype: float64


In [0]:
# Train Error
y_pred = [guess] * len(y_train)
mae = mean_absolute_error(y_train,y_pred)
print(f'Train Error: ${mae:.2f}')

Train Error: $601.90


In [0]:
print('Mean Baseline (using 0 features):')
guess = y_test.mean()
print('$',(guess))

Mean Baseline (using 0 features):
$ 3587.0546750721733


In [0]:
# Test Error
y_pred = [guess] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'Test Error: ${mae:.2f}')

Test Error: $1200.87


# Run linear regression model

In [0]:
# Step One we've already imported the appropriate estimator above^
# 2. Instantiate this class with LinearRegression():
model = LinearRegression()

In [0]:
# 3. Arrange X features matrices (already did y target vectors above^)
features = ['total_perks', 'bathroom_baths_ratio']
target = 'price'
X_train = df[features]
X_test = df[target]
y_train = df[features]
y_test = df[target]
print(f'Linear Regression, dependent on: {features}')

Linear Regression, dependent on: ['total_perks', 'bathroom_baths_ratio']


In [0]:
model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [0]:
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print('Train error:', mae)

Train error: 1.5013902623943394e-13


#  Get the model's coefficients and intercept.

In [0]:
model.intercept_, model.coef_

(array([-9.09494702e-13, -4.44089210e-16]),
 array([[ 1.00000000e+00,  3.71975813e-16],
        [-5.22498378e-20,  1.00000000e+00]]))

# $y = \beta_0 + \beta_1x_1 + \beta_2x_2$


In [0]:
beta0 = model.intercept_
beta1, beta2 = model.coef_
print(f'y = {beta0} + {beta1}x1 + {beta2}x2')

y = [-9.09494702e-13 -4.44089210e-16] + [1.00000000e+00 3.71975813e-16]x1 + [-5.22498378e-20  1.00000000e+00]x2


In [0]:
# Practice writing it out like this:
print('Intercept', model.intercept_)
coefficients = pd.Series(model.coef_, features)
print(coefficients.to_string())

# RMSE, MAE, and 𝑅2

In [0]:
# Root Mean square error:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [0]:
mse_train = mean_squared_error(y_train, y_pred)
rmse_train = np.sqrt(mse_train)

# mean absolute error:
mae_train = mean_absolute_error(y_train, y_pred)

# R squared:
r2_train = r2_score(y_train, y_pred)

print('Mean Squared Error:', mse_train)
print('Root Mean Squared Error:', rmse_train)
print('Mean Absolute Error:', mae_train)
print('R^2:', r2_train)

Mean Squared Error: 1.1713597545839637e-25
Root Mean Squared Error: 3.4225133375692836e-13
Mean Absolute Error: 1.5013902623943394e-13
R^2: 1.0


In [0]:
# root mean square error:
mse_test = mean_squared_error(y_test, y_pred)
rmse_test = np.sqrt(mse_test)

# mean absolute error:
mae_test = mean_absolute_error(y_test, y_pred)

# R squared:
r2_test = r2_score(y_test, y_pred)

print('Mean Squared Error:', mse_test)
print('Root Mean Squared Error:', rmse_test)
print('Mean Absolute Error:', mae_test)
print('R^2:', r2_test)

In [0]:
y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred)**(.5)
print(f'Test RMSE: ${rmse:,.2f}')

mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE: ${mae:,.2f}')

R2 = model.score(X_test, y_test)
print(f'Test R^2: {R2:,.4f}')

In [0]:
print('Mean Squared Error:', mse_test)
print('Root Mean Squared Error:', rmse_test)
print('Mean Absolute Error:', mae_test)
print('R^2:', r2_test)